# 使用全链接神经网络进行训练

## 1 导入数据

In [20]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import mean_squared_error

In [21]:
train = pd.read_csv("../../kaggle/LosAngelesHousePricesForecast/data/train_1.csv")
if 'Unnamed: 0' in train.columns: 
    train = train.drop(['Unnamed: 0'], axis=1)
y = train["SalePrice"]
train1 = train.drop(["Id", "SalePrice"], axis=1)
X = pd.get_dummies(train1).reset_index(drop=True)
X_train, X_test, y_train, y_test = tts(X,y,test_size=0.2,random_state = 123)   

In [22]:
X.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,0,0,0,1,0,0,0,0,1,0
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,0,0,0,1,0,0,0,0,1,0
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,0,0,0,1,0,0,0,0,1,0
3,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,0,0,0,1,1,0,0,0,0,0
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,0,0,0,1,0,0,0,0,1,0


### 评估函数

In [23]:
def benchmark(model):
    pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, pred))
    logrmse = np.sqrt(mean_squared_error(np.log(y_test), np.log(pred)))
    print("RMSE: {} \nLOGRMSE: {}".format(rmse, logrmse))
    return rmse, logrmse

## 2 模型训练

In [24]:
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import metrics
from keras import backend as K
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [25]:
keras.__version__

'2.2.4'

In [26]:
tf.__version__

'1.12.0'

In [27]:
def create_model():
    model = Sequential()
    model.add(Dense(10, input_dim=X_train.shape[1], activation="relu"))
    model.add(Dense(300,activation="relu")) 
    model.add(Dropout(0.2))
    model.add(Dense(50, activation="relu"))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [28]:
model = KerasRegressor(build_fn=create_model)

In [36]:
batch_size = [16, 32, 48, 64, 80, 96]
epochs = [500, 1000, 2000, 3000, 3500, 4000]
# ?RandomizedSearchCV
param_grid = dict(batch_size=batch_size,epochs=epochs)
rscv = RandomizedSearchCV(estimator=model, param_distributions=param_grid, verbose=0, n_jobs=-1,random_state=123)
result = rscv.fit(X_train, y_train)
for mean,stdev,param in zip(result.cv_results_['mean_test_score'],
                           result.cv_results_['std_test_score'],
                           result.cv_results_['params']):
    print("mean:{} stdev:{} param:{}".format(mean,stdev,param))
print("Best score: {} Best param:{}".format(result.best_score_, result.best_params_))

# grid = GridSearchCV(estimator=model, param_grid=param_grid, verbose=0, n_jobs=-1)
# grid_res = grid.fit(X_train, y_train)
# for mean,stdev,params in zip(grid_res.cv_results_['mean_test_score'],
#                            grid_res.cv_results_['std_test_score'],
#                            grid_res.cv_results_['params']):
#     print("mean:{} stdev:{} params:{}".format(mean,stdev,params))
# print("Best score: {} Best params:{}".format(grid_res.best_score_, grid_res.best_params_))

KeyboardInterrupt: 

In [22]:
# Best score: -951423830.0616438 Best params:{'batch_size': 48, 'epochs': 4000}

In [30]:
history = model.fit(X_train, y_train, epochs=500, batch_size=32, verbose=0)

KeyboardInterrupt: 

In [24]:
benchmark(model)

RMSE: 30255.047113464894 
LOGRMSE: 0.13724170239696631


(30255.047113464894, 0.13724170239696631)

In [8]:
def create_model():
    model = Sequential()
    model.add(Dense(10, input_dim=X_train.shape[1], activation="relu"))
    model.add(Dense(300, activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [9]:
model = create_model()

In [12]:
y_log_train = np.log(y_train)

In [13]:
model.fit(X_train, y_train, epochs=3500, batch_size=32, verbose=0)
# 3500,32 0.128
# 4000,32,0.131
# 3000,32,0.131
# 3200,32,0.241


In [14]:
benchmark(model)

RMSE: 30139.91903364521 
LOGRMSE: 0.13884507160088366


(30139.91903364521, 0.13884507160088366)

In [15]:
log_model = create_model()

In [16]:
log_model.fit(X_train, y_log_train, epochs=3500, batch_size=32, verbose=0)

In [17]:
def benchmark2(model):
    pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, np.exp(pred)))
    logrmse = np.sqrt(mean_squared_error(np.log(y_test), pred))
    print("RMSE: {} \nLOGRMSE: {}".format(rmse, logrmse))
    return rmse, logrmse

In [19]:
benchmark2(log_model)

RMSE: 171274.25957772543 
LOGRMSE: 1.9216737769849495


(171274.25957772543, 1.9216737769849495)

In [14]:
# import matplotlib.pyplot as plt 
# %matplotlib inline
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.show()

In [20]:
model=create_model()
history=model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=3350,batch_size=32,verbose=0)

In [21]:
benchmark(model)

RMSE: 31665.02579389985 
LOGRMSE: 0.14228086659366299


(31665.02579389985, 0.14228086659366299)